# Multi-Agent Orchestration with Semantic Kernel

Welcome to the fascinating world of multi-agent orchestration! In this notebook, we'll explore how multiple AI agents can work together to solve complex problems through different orchestration patterns.



## Package References and Configuration

First, let's install the required NuGet packages for multi-agent orchestration and create our kernel

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.67.1"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Kernel = Microsoft.SemanticKernel.Kernel;

Kernel CreateKernel()
{
    var builder = Kernel.CreateBuilder();

    var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId, embeddingEndpoint, embeddingApiKey) = Settings.LoadFromFile();

    builder.AddAzureOpenAIChatCompletion(model, azureEndpoint, apiKey);
    var kernel = builder.Build();

    return kernel;
}

var kernel = CreateKernel();

Installed Packages Microsoft.SemanticKernel, 1.67.1

## Configuration and Settings

Load our AI configuration settings:

## Using Statements and Dependencies

Import all necessary namespaces for multi-agent orchestration:

In [2]:
#r "nuget: Microsoft.SemanticKernel.Agents.Core, 1.67.1"
#r "nuget: Microsoft.SemanticKernel.Agents.Orchestration, 1.67.1-preview"
#r "nuget: Microsoft.SemanticKernel.Agents.Runtime.InProcess, 1.67.1-preview"

Installed Packages Microsoft.SemanticKernel.Agents.Core, 1.67.1 Microsoft.SemanticKernel.Agents.Orchestration, 1.67.1-preview Microsoft.SemanticKernel.Agents.Runtime.InProcess, 1.67.1-preview

because we are using a preview and (still) experimental version, we need to diable the warning:
*error SKEXP0110: 'Microsoft.SemanticKernel.Agents.Orchestration.Concurrent.ConcurrentOrchestration' is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.*

In [3]:
#pragma warning disable SKEXP0110

In [4]:
// Core Semantic Kernel imports
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;

// System imports for orchestration
using System;
using System.Collections.Generic;
using System.Threading.Tasks;
using System.Threading;
using System.Linq;
using System.Collections.Concurrent;
using System.Text;
using Microsoft.DotNet.Interactive;

// Kernel alias to avoid conflicts
using Kernel = Microsoft.SemanticKernel.Kernel;

Console.WriteLine("All dependencies imported successfully! 🎯");

All dependencies imported successfully! 🎯


## Helper functions



In [5]:
// Helper function to display agent responses
void DisplayResponse(string agentName, string response, string emoji = "🤖")
{
    Console.WriteLine($"\n{emoji} **{agentName}:**");
    Console.WriteLine($"   {response}");
    Console.WriteLine(new string('-', 50));
}

// Helper function to display section headers
void DisplaySection(string title, string emoji = "🎯")
{
    Console.WriteLine($"\n{emoji} {title}");
    Console.WriteLine(new string('=', title.Length + 4));
}

Console.WriteLine("🛠️ Helper functions created successfully!");

🛠️ Helper functions created successfully!


## Simple Concurrent Orchestration

Let's start with our first orchestration pattern: **Concurrent Orchestration**

### Scenario: Restaurant Kitchen Team 🍽️

Imagine a busy restaurant kitchen where multiple chefs work simultaneously on different parts of a meal:

In [6]:
#pragma warning disable SKEXP0110

using Microsoft.SemanticKernel.Agents.Orchestration;
using Microsoft.SemanticKernel.Agents.Orchestration.Concurrent;
using Microsoft.SemanticKernel.Agents.Runtime.InProcess;


DisplaySection("CONCURRENT ORCHESTRATION - Restaurant Kitchen Team", "🍽️");

// Create our restaurant kitchen team
var sousChef = new ChatCompletionAgent
    {
        Name = "SousChef",
        Description = "You are a sous chef specializing in appetizers and salads",
        Instructions = "Be creative and describe the dish preparation briefly",
        Kernel = kernel,
        Arguments = new KernelArguments(new OpenAIPromptExecutionSettings
        {
            MaxTokens = 500,
            Temperature = 0.7
        })
    };

var mainChef = new ChatCompletionAgent
    {
        Name = "MainChef",
        Description = "You are a main course chef specializing in proteins and hearty dishes",
        Instructions = "Be creative and describe the cooking process briefly",
        Kernel = kernel
    };


var pastryChef = new ChatCompletionAgent
    {
        Name = "PastryChef",
        Description = "You are a pastry chef specializing in desserts and sweet treats",
        Instructions = "Be creative and describe the dessert preparation briefly",
        Kernel = kernel
    };
    
// Define the orchestration
var orchestration =
    new ConcurrentOrchestration(sousChef, mainChef, pastryChef);

// Start the runtime
var runtime = new InProcessRuntime();

await runtime.StartAsync();


OrchestrationResult<string[]> result = await orchestration.InvokeAsync("Create a delicious israeli meal", runtime);

string[] output = await result.GetValueAsync();
Console.WriteLine($"\n# RESULT:\n{string.Join("\n\n", output.Select(text => $"{text}"))}");

await runtime.RunUntilIdleAsync();


🍽️ CONCURRENT ORCHESTRATION - Restaurant Kitchen Team

# RESULT:
Prepare a mouthwatering Israeli feast with a medley of flavors! Start with **homemade hummus**: blend chickpeas, tahini, garlic, lemon juice, olive oil, and a pinch of cumin until creamy. Drizzle with olive oil, sprinkle paprika, and garnish with parsley.

Next, craft **falafel**: soak chickpeas overnight, then blend with onion, garlic, parsley, cilantro, cumin, and coriander. Form into balls and deep fry until golden and crispy. Pair with a fresh **Israeli salad** made of diced cucumbers, tomatoes, red onion, and parsley, dressed with olive oil, fresh lemon juice, and salt.

For the main dish, serve **shakshuka**: sauté onions, garlic, bell peppers, and tomatoes with a blend of paprika, cumin, and chili flakes. Crack eggs into the simmering tomato mixture and cook until the whites are set but yolks remain runny. Garnish with fresh cilantro and serve with warm, fluffy pita bread.

Round it out with a side of **labneh** (

### Adding the inner interaction and handling messages

In [7]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

var orchestrationWithInnerCommunication =
    new ConcurrentOrchestration(sousChef, mainChef, pastryChef)
    {
        ResponseCallback = (ChatMessageContent response)=>
            {   
                DisplayResponse(response.AuthorName, response.Content);
                return ValueTask.CompletedTask;
            }
    };

// Start the runtime
var runtime = new InProcessRuntime();

await runtime.StartAsync();


OrchestrationResult<string[]> result = await orchestrationWithInnerCommunication.InvokeAsync("Create a delicious israeli meal", runtime);

string[] output = await result.GetValueAsync();
//Console.WriteLine($"\n# RESULT:\n{string.Join("\n\n", output.Select(text => $"{text}"))}");

await runtime.RunUntilIdleAsync();




🤖 **MainChef:**
   To whip up a delicious Israeli meal, start by preparing *hummus*, the creamy and quintessential dip. Blend chickpeas, tahini, garlic, lemon juice, olive oil, and a sprinkle of cumin until velvety smooth, then drizzle it with more olive oil and garnish with smoked paprika.

For your main dish, make *shakshuka*, a vibrant one-pan delight. Sauté onions, garlic, bell peppers, and tomatoes in olive oil, spicing it up with paprika, cumin, and a pinch of chili flakes. Crack fresh eggs into the bubbling tomato sauce and let them gently poach, their yolks golden and runny. Serve with warm, fluffy pita bread to scoop up all the goodness.

On the side, toss together a bright *Israeli salad*—finely dice cucumbers, tomatoes, red onion, and parsley, then dress it lightly with olive oil, lemon juice, and salt for maximum freshness.

Finally, round out the meal with a serving of *malabi*, a delicate rose-water-infused milk pudding, topped with crushed pistachios and a drizzle of sy

## Sequential Orchestration

Now let's explore **Sequential Orchestration** where agents work in a specific order.

### Scenario: News Editorial Team 📰

In a newsroom, articles go through a specific workflow: Writing → Editing

In [8]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

using Microsoft.SemanticKernel.Agents.Orchestration.Sequential;

DisplaySection("SEQUENTIAL ORCHESTRATION - News Editorial Team", "📰");

// Create our news editorial team
var writer = new ChatCompletionAgent
    {
        Name = "Writer",
        Description = "You are a news writer",
        Instructions = "Take the research and write a compelling news article. Keep it engaging and informative",
        Kernel = kernel
    };


var editor = new ChatCompletionAgent
    {
        Name = "Editor",
        Description = "You are a news editor",
        Instructions = 
        """
        Review the given draft article and imporve clarity, correct grammer and shorten it.
        Output the final improved copy as a single text block.
        """,
        Kernel = kernel
    };
    
// Define the orchestration
var orchestration =
    new SequentialOrchestration(writer, editor)
    {
        ResponseCallback = (ChatMessageContent response)=>
            {   
                DisplayResponse(response.AuthorName, response.Content);
                return ValueTask.CompletedTask;
            }
    };

// Start the runtime
var runtime = new InProcessRuntime();

await runtime.StartAsync();


OrchestrationResult<string> result = await orchestration.InvokeAsync("Write an article on Semantic Kernel Agents", runtime);

string output = await result.GetValueAsync();
Console.WriteLine($"\n# RESULT:\n{output}");

await runtime.RunUntilIdleAsync();


📰 SEQUENTIAL ORCHESTRATION - News Editorial Team

🤖 **Writer:**
   **Unlocking the Power of Semantic Kernel Agents: The Future of AI-Powered Task Management**

*By [Your Name]*  
*Date: [Enter Date]*

In the ever-evolving frontier of artificial intelligence, Semantic Kernel Agents are emerging as revolutionary tools poised to change the way we manage and interact with complex information and tasks. Blurring the line between human-like understanding and computational precision, these agents are redefining the modern approach to automation, personalization, and decision-making.

## What Are Semantic Kernel Agents?

At their core, Semantic Kernel Agents utilize cutting-edge AI principles to merge natural language processing (NLP) with semantic comprehension, enabling them to go beyond the surface level of simple command response. By tapping into vast knowledge bases, these agents can process and synthesize vast quantities of data, recognize nuanced relationships between concepts, and del

## Group Chat Orchestration

Let's explore **Group Chat Orchestration** where agents collaborate in a discussion.

### Scenario: Medical Consultation Team 🏥

A medical consultation where specialists discuss a patient case together:

In [9]:
DisplaySection("GROUP CHAT ORCHESTRATION - Medical Consultation Team", "🏥");

// Create our medical consultation team
var generalDoctor = new ChatCompletionAgent
    {
        Name = "GeneralDoctor",
        Description = "You are a general practitioner",
        Instructions = 
        """
        Provide overall medical assessment and coordinate with specialists. 
        Be professional and concise.
        You are part of a team of doctors and you need to take their input and evaluate your answer accordingly, so always summarize the input you received.
        """,
        Kernel = kernel
    };

 
var cardiologist = new ChatCompletionAgent
    {
        Name = "Cardiologist",
        Description = "You are a heart specialist",
        Instructions = 
        """
        Focus on cardiovascular aspects of the case. 
        Provide expert cardiac insights.
        You are part of a team of doctors and you need to take their input and evaluate your answer accordingly, so always summarize the input you received.
        """,
        Kernel = kernel
    };
    
var neurologist = new ChatCompletionAgent
    {
        Name = "Nutritionist",
        Description = "You are a neurologist",
        Instructions = 
        """
        Focus on neurological aspects of the case. 
        Provide expert brain and nervous system insights.
        You are part of a team of doctors and you need to take their input and evaluate your answer accordingly, so always summarize the input you received.
        """,
        Kernel = kernel
    };



// Create a shared chat history for the group consultation
var groupChatHistory = new ChatHistory();



🏥 GROUP CHAT ORCHESTRATION - Medical Consultation Team


In [10]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

using Microsoft.SemanticKernel.Agents.Orchestration.GroupChat;

var orchestration =
    new GroupChatOrchestration(
        new RoundRobinGroupChatManager()  { MaximumInvocationCount = 5  },
        generalDoctor,
        cardiologist,
        neurologist)
    {       
        ResponseCallback =  chatMessage => 
        { 
            groupChatHistory.Add(chatMessage);
            return ValueTask.CompletedTask;
        }
    };

// Start the runtime
var runtime = new InProcessRuntime();
await runtime.StartAsync();


OrchestrationResult<string> result = await orchestration.InvokeAsync("Everytime i workout my breath makes funny sounds. Tell me what to do", runtime);
string text = await result.GetValueAsync();
Console.WriteLine($"\n# RESULT: {text}");

await runtime.RunUntilIdleAsync();

Console.WriteLine("\n\nORCHESTRATION HISTORY");
foreach (ChatMessageContent message in groupChatHistory)
{
    DisplayResponse(message.AuthorName, message.Content);
}


# RESULT: ### Cardiovascular Summary & Next Steps for Breathing Sounds During Exercise

From a cardiologist's perspective, while the breathing sounds during workouts might initially seem respiratory, exercise-related symptoms can occasionally signal **cardiovascular pathology**, especially if paired with systemic indicators like chest discomfort, palpitations, or fatigue. The collaborative findings from all specialties suggest targeting respiratory evaluations first but remaining vigilant for heart-related concerns.

---

### **Cardiovascular Evaluation Plan**
#### Symptoms to Watch for Cardiac Involvement:
1. **Shortness of Breath That Persists Post-Exercise** or worsens disproportionately compared to exertion level.
2. **Dizziness, Chest Pressure/Pain, Fainting**, or rapid heart rate during workouts.
3. **Fatigue or Lower Extremity Swelling** indicating potential volume overload (a subtle sign of cardiac dysfunction).

#### Cardiac Diagnostic Suggestions:
- **Electrocardiogram (ECG)

## Custom Orchestration

Finally, let's explore **Custom Orchestration** - controlling the agents selection and termincation logic.

### Scenario: Hospital's Tumor Board 🎵

Hospital's tumor board is reviewing a complex cancer case:

In [11]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents.Orchestration.GroupChat;
using Microsoft.SemanticKernel.ChatCompletion;

class TumorBoardLeader : GroupChatManager
{
    string _boardCase;
    IChatCompletionService _chatCompletion;

    public TumorBoardLeader(string boardCase, IChatCompletionService chatCompletion)
    {
        _boardCase = boardCase;
        _chatCompletion = chatCompletion;
    }

    private static class Prompts
    {
        public static string Termination(string boardCase) =>
            $"""
                You are a leader of the hospital tumor board. The case the board is working on is : '{boardCase}'. 
                You need to determine if the discussion has reached a conclusion and if all next steps were listed.
                if no artifacts were produced yet and you don't have knowledge of them yet, then continue the dicsussion and planning.
                
                If you would like to end the discussion, please respond with True. Otherwise, respond with False.
                """;

        public static string Selection(string boardCase, string participants) =>
            $"""
                Tou are a leader of the hospital tumor board. The case the board is working on is : '{boardCase}'. 
                You need to select the next team member to speak and suggest a plan or next steps. 
                Here are the roles and descriptions of the participants: 
                {participants}\n
                Please respond with only the role of the participant you would like to select.
                """;

        public static string Filter(string boardCase) =>
            $"""
                ou are a leader of the hospital tumor board. The case the board is working on is : '{boardCase}'. 
                You have just concluded the discussion and team work. 
                Please summarize the discussion and provide the final artifacts.
                """;
    }

    /// <inheritdoc/>
    public override ValueTask<GroupChatManagerResult<string>> FilterResults(ChatHistory history, CancellationToken cancellationToken = default) =>
        this.GetResponseAsync<string>(history, Prompts.Filter(_boardCase), cancellationToken);

    /// <inheritdoc/>
    public override ValueTask<GroupChatManagerResult<string>> SelectNextAgent(ChatHistory history, GroupChatTeam team, CancellationToken cancellationToken = default) =>
        this.GetResponseAsync<string>(history, Prompts.Selection(_boardCase, team.FormatList()), cancellationToken);

    /// <inheritdoc/>
    public override ValueTask<GroupChatManagerResult<bool>> ShouldRequestUserInput(ChatHistory history, CancellationToken cancellationToken = default) =>
        ValueTask.FromResult(new GroupChatManagerResult<bool>(false) { Reason = "The AI group chat manager does not request user input." });

    /// <inheritdoc/>
    public override async ValueTask<GroupChatManagerResult<bool>> ShouldTerminate(ChatHistory history, CancellationToken cancellationToken = default)
    {
        GroupChatManagerResult<bool> result = await base.ShouldTerminate(history, cancellationToken);
        if (!result.Value)
        {
            result = await this.GetResponseAsync<bool>(history, Prompts.Termination(_boardCase), cancellationToken);
        }
        return result;
    }

    private async ValueTask<GroupChatManagerResult<TValue>> GetResponseAsync<TValue>(ChatHistory history, string prompt, CancellationToken cancellationToken = default)
    {
        OpenAIPromptExecutionSettings executionSettings = new() { ResponseFormat = typeof(GroupChatManagerResult<TValue>) };
        ChatHistory request = [.. history, new ChatMessageContent(AuthorRole.System, prompt)];
        ChatMessageContent response = await _chatCompletion.GetChatMessageContentAsync(request, executionSettings, kernel: null, cancellationToken);
        string responseText = response.ToString();
        
        return
            JsonSerializer.Deserialize<GroupChatManagerResult<TValue>>(responseText) ??
            throw new InvalidOperationException($"Failed to parse response: {responseText}");
    }
}

In [12]:
DisplaySection("CUSTOM ORCHESTRATION - Hospital's tumor board is reviewing a complex cancer case", "🎵");

var oncologistAgent = new ChatCompletionAgent
{
    Name = "OncologistAgent",
    Instructions = 
        """
        You are a medical oncologist participating in a multidisciplinary discussion about a cancer patient's treatment. 
        Your role is to evaluate drug-based treatment options, considering cancer stage, biomarkers, and mutation data. 
        You collaborate with a radiologist and a pathologist. Be open to revising your recommendation based on their input.
        Only suggest treatments after reviewing all available clinical and diagnostic information.

        You are the final caller, the decision comes from you and you decide if more inputs are needed from the others.
        The final call and decision is yours! the result of the discussion should come from you so when you think the discussion is over, say it.
        """,
    Description = "A medical oncologist who suggests and revises treatment plans based on pathology and imaging insights.",
    Kernel = kernel,
    Arguments = new KernelArguments(new PromptExecutionSettings{ FunctionChoiceBehavior = FunctionChoiceBehavior.Auto(),})
};

var pathologistAgent = new ChatCompletionAgent
{
    Name = "PathologistAgent",
    Instructions = 
        """
        You are a clinical pathologist providing molecular and genetic analysis of a patient's tumor. 
        Share insights about mutations, biomarkers, and how they might affect treatment effectiveness. 
        Collaborate with the oncologist and radiologist. Suggest when additional lab tests may be needed.
        The final call and decision is of the oncologist!
        """,
    Description = "A clinical pathologist who interprets molecular and genetic data relevant to treatment decisions.",
    Kernel = kernel,
    Arguments = new KernelArguments(new PromptExecutionSettings{ FunctionChoiceBehavior = FunctionChoiceBehavior.Auto(),})

};

var radiologistAgent = new ChatCompletionAgent
{
    Name = "RadiologistAgent",
    Instructions = 
        """
        You are a radiologist interpreting imaging results for a cancer patient. 
        Point out any concerns such as lymph node involvement or metastasis. 
        Inform how imaging findings affect staging and treatment planning. Collaborate with the oncologist and pathologist.
        The final call and decision is of the oncologist!
        """,
    Description = "A radiologist who provides imaging-based insights and identifies implications for treatment planning.",
    Kernel = kernel,
    Arguments = new KernelArguments(new PromptExecutionSettings{ FunctionChoiceBehavior = FunctionChoiceBehavior.Auto(),})
};

// Create a shared chat history for the group consultation
var tumorBoardHistory = new ChatHistory();


🎵 CUSTOM ORCHESTRATION - Hospital's tumor board is reviewing a complex cancer case


In [13]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

using Microsoft.SemanticKernel.Agents.Orchestration;
using Microsoft.SemanticKernel.Agents.Runtime.InProcess;

string input =
    """
    Patient: Female, 55 years old  
    Diagnosis: Invasive ductal carcinoma, grade 3  
    Tumor Size: 2.8 cm  
    Stage: T2N0M0 (early-stage)  
    Hormone Receptors: ER+, PR+, HER2-  
    Ki-67: 35% (high proliferation)  
    Genetic Testing: BRCA-negative  
    Genomic Profile: PIK3CA mutation detected  
    Imaging: Mammogram and MRI confirm localized tumor, but one axillary lymph node appears suspicious (not biopsied yet)  
    Comorbidities: Mild hypertension  

    Please collaborate and determine the optimal treatment plan for this patient. Consider systemic therapies, genetic markers, and imaging findings. 
    Discuss whether further testing is needed, and iterate based on each other's input.
    The final call and decision is of the oncologist.
    """;


var chatCompletionService = kernel.GetRequiredService<IChatCompletionService>();

var tumorBoardOrchestration =
   new GroupChatOrchestration(
       new TumorBoardLeader(input, chatCompletionService),
       
       oncologistAgent, 
       pathologistAgent, 
       radiologistAgent)
   {
    ResponseCallback =  chatMessage => 
       { 
            DisplayResponse(chatMessage.AuthorName, chatMessage.Content);
            tumorBoardHistory.Add(chatMessage);
            return ValueTask.CompletedTask;  
       }
   };
// Start the runtime
InProcessRuntime runtime = new();
await runtime.StartAsync();



OrchestrationResult<string> result = await tumorBoardOrchestration.InvokeAsync(input, runtime);
string text = await result.GetValueAsync();
Console.WriteLine($"\n# RESULT: {text}");

await runtime.RunUntilIdleAsync();

Console.WriteLine("\n\nORCHESTRATION HISTORY");
foreach (ChatMessageContent message in tumorBoardHistory)
{
    DisplayResponse(message.AuthorName, message.Content);
}


🤖 **RadiologistAgent:**
   Here is my input regarding the imaging findings and overall scenario based on the available data:

### Imaging Concerns:
1. **Axillary Lymph Node Involvement**
   - One axillary lymph node appears **suspicious** on imaging (mammogram/MRI). This could suggest potential spread to the regional lymph nodes. Since the staging currently indicates **N0**, a biopsy or fine-needle aspiration of the suspicious lymph node is essential to confirm or rule out nodal involvement.
   - If the lymph node is positive for metastatic carcinoma, the staging would change from **T2N0M0** to **T2N1M0**, which would upgrade the disease stage and influence systemic therapy and surgical planning.

### Implications of Imaging on Treatment:
- If the lymph node is involved:
  - This would push the treatment strategy toward systemic therapy such as chemotherapy (e.g., anthracycline- or taxane-based regimens) and possibly regional nodal radiotherapy.
  - Surgical planning might include axi

## Human-in-the-Loop (HITL) with Group Chat Orchestration



Now let's explore adding **Human-in-the-Loop** to our agent orchestration.



### Scenario: Medical Team with Human Approval Gate 🏥🚦



A medical consultation where the team's recommendation requires human approval before being finalized:

In [14]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001

using Microsoft.SemanticKernel.Agents.Orchestration.GroupChat;

// Custom GroupChatManager with Human Approval
class HumanApprovalManager : GroupChatManager
{
    private bool _approvalRequested = false;    

    public override async ValueTask<GroupChatManagerResult<bool>> ShouldTerminate(
        ChatHistory history, 
        CancellationToken cancellationToken = default)
    {
        // Check base termination (max invocations)
        var baseResult = await base.ShouldTerminate(history, cancellationToken);        

        // If agents think they're done (or max invocations reached), request human approval
        if (baseResult.Value && !_approvalRequested)
        {
            _approvalRequested = true;            

            Console.WriteLine("\n" + new string('=', 60));
            Console.WriteLine("🚦 HUMAN APPROVAL REQUIRED");
            Console.WriteLine(new string('=', 60));
            Console.WriteLine("\nThe medical team has completed their discussion.");
            Console.WriteLine("\n📋 Summary of Discussion:");            

            // Show last 3 messages
            foreach (var msg in history.TakeLast(3))
            {
                Console.WriteLine($"\n{msg.AuthorName}: {msg.Content?.Substring(0, Math.Min(150, msg.Content?.Length ?? 0))}...");
            }            

            Console.WriteLine("\n" + new string('-', 60));
            Console.WriteLine("\nDo you approve this recommendation?");
            Console.WriteLine("  Type 'yes' to approve and finalize");
            Console.WriteLine("  Type 'no' to reject and provide feedback");
            Console.Write("\nYour decision: ");            

            var input = Console.ReadLine()?.Trim().ToLower();            

            if (input == "yes" || input == "y")
            {
                Console.WriteLine("\n✅ APPROVED - Discussion will conclude");
                Console.WriteLine(new string('=', 60));

                return new GroupChatManagerResult<bool>(true) 
                { 
                    Reason = "Human approved the recommendation" 
                };

            }
            else
            {
                Console.WriteLine("\n❌ REJECTED - Please provide feedback");
                Console.Write("Your feedback for the team: ");
                var feedback = Console.ReadLine();                

                // Add human feedback to history
                history.AddUserMessage($"[HUMAN REVIEWER FEEDBACK]: {feedback}. Please revise your recommendation.");                

                Console.WriteLine("\n🔄 Discussion will continue with your feedback...");
                Console.WriteLine(new string('=', 60));                

                _approvalRequested = false; // Reset for next approval cycle

                return new GroupChatManagerResult<bool>(false) 
                { 
                    Reason = "Human rejected - continuing with feedback" 
                };
            }
        }        

        return baseResult;
    }
}

Console.WriteLine("✅ HumanApprovalManager created!");

Error: (7,7): error CS0534: 'HumanApprovalManager' does not implement inherited abstract member 'GroupChatManager.SelectNextAgent(ChatHistory, GroupChatTeam, CancellationToken)'
(7,7): error CS0534: 'HumanApprovalManager' does not implement inherited abstract member 'GroupChatManager.ShouldRequestUserInput(ChatHistory, CancellationToken)'
(7,7): error CS0534: 'HumanApprovalManager' does not implement inherited abstract member 'GroupChatManager.FilterResults(ChatHistory, CancellationToken)'

### Run the HITL Orchestration



Now let's create a medical consultation with human approval:

In [15]:
DisplaySection("HITL ORCHESTRATION - Medical Team with Human Approval", "🏥🚦");



// Create medical team agents

var generalDoctor = new ChatCompletionAgent
{
    Name = "GeneralDoctor",
    Description = "You are a general practitioner",
    Instructions = 
    """
    Provide overall medical assessment and coordinate with specialists. 
    Be professional and concise.
    When you receive human feedback, address it seriously and revise your recommendations.
    Summarize the input from other doctors.
    """,
    Kernel = kernel
};



var cardiologist = new ChatCompletionAgent
{
    Name = "Cardiologist",
    Description = "You are a heart specialist",

    Instructions = 

    """

    Focus on cardiovascular aspects of the case. 

    Provide expert cardiac insights.

    If human feedback mentions cardiac concerns, address them specifically.

    """,

    Kernel = kernel

};



var nutritionist = new ChatCompletionAgent

{

    Name = "Nutritionist",

    Description = "You are a nutritionist",

    Instructions = 

    """

    Focus on dietary and lifestyle aspects of the case.

    Provide nutrition-based recommendations.

    Incorporate human feedback into dietary plans.

    """,

    Kernel = kernel

};



// Create chat history

var hitlChatHistory = new ChatHistory();



// Create orchestration with human approval

var hitlOrchestration = new GroupChatOrchestration(

    new HumanApprovalManager() { MaximumInvocationCount = 6 },  // Max 6 agent turns before requiring approval

    generalDoctor,

    cardiologist,

    nutritionist)

{

    ResponseCallback = chatMessage => 

    {

        DisplayResponse(chatMessage.AuthorName, chatMessage.Content);

        hitlChatHistory.Add(chatMessage);

        return ValueTask.CompletedTask;

    }

};



// Start runtime

var runtime = new InProcessRuntime();

await runtime.StartAsync();



// Run the consultation

var result = await hitlOrchestration.InvokeAsync(

    "Patient has high blood pressure and high cholesterol. Recommend a treatment plan.",

    runtime);



string finalRecommendation = await result.GetValueAsync();



await runtime.RunUntilIdleAsync();



Console.WriteLine("\n" + new string('=', 60));

Console.WriteLine("📋 FINAL APPROVED RECOMMENDATION:");

Console.WriteLine(new string('=', 60));

Console.WriteLine(finalRecommendation);

Error: (80,29): error SKEXP0110: 'Microsoft.SemanticKernel.Agents.Orchestration.GroupChat.GroupChatOrchestration' is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
(82,9): error CS0246: The type or namespace name 'HumanApprovalManager' could not be found (are you missing a using directive or an assembly reference?)
(92,5): error SKEXP0110: 'Microsoft.SemanticKernel.Agents.Orchestration.AgentOrchestration<string, string>.ResponseCallback' is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
(92,5): error SKEXP0110: 'Microsoft.SemanticKernel.Agents.Orchestration.AgentOrchestration<string, string>.ResponseCallback.init' is for evaluation purposes only and is subject to change or removal in future updates. Suppress this diagnostic to proceed.
(118,20): error SKEXP0110: 'Microsoft.SemanticKernel.Agents.Orchestration.AgentOrchestration<string, s

## Alternative HITL Pattern: Periodic Check-ins



Here's another pattern where humans can provide input periodically during the discussion:

In [16]:
#pragma warning disable SKEXP0110
#pragma warning disable SKEXP0001



// Manager that asks for human input every N messages

class PeriodicHumanInputManager : GroupChatManager

{

    private readonly int _checkInFrequency;

    

    public PeriodicHumanInputManager(int checkInEveryNMessages = 3)

    {

        _checkInFrequency = checkInEveryNMessages;

    }

    

    public override ValueTask<GroupChatManagerResult<bool>> ShouldRequestUserInput(

        ChatHistory history,

        CancellationToken cancellationToken = default)

    {

        // Request input every N messages

        if (history.Count > 0 && history.Count % _checkInFrequency == 0)

        {

            Console.WriteLine($"\n💬 Human check-in requested after {_checkInFrequency} messages");

            Console.WriteLine("Type your input (or press Enter to continue without input):");

            Console.Write("> ");

            

            return ValueTask.FromResult(

                new GroupChatManagerResult<bool>(true) 

                { 

                    Reason = "Periodic human check-in" 

                });

        }

        

        return ValueTask.FromResult(

            new GroupChatManagerResult<bool>(false));

    }

}



Console.WriteLine("✅ PeriodicHumanInputManager created!");

Console.WriteLine("This manager will request human input every 3 agent messages.");

Error: (8,7): error CS0534: 'PeriodicHumanInputManager' does not implement inherited abstract member 'GroupChatManager.SelectNextAgent(ChatHistory, GroupChatTeam, CancellationToken)'
(8,7): error CS0534: 'PeriodicHumanInputManager' does not implement inherited abstract member 'GroupChatManager.FilterResults(ChatHistory, CancellationToken)'

## Key HITL Patterns



We've explored two main HITL patterns:



### 1. **Approval Gates** (`ShouldTerminate`)

- Human approval required before finalizing

- Can reject and provide feedback

- Agents revise based on feedback

- **Best for**: Final decisions, critical actions



### 2. **Periodic Check-ins** (`ShouldRequestUserInput`)

- Human can guide during discussion

- Proactive intervention

- Course correction in real-time

- **Best for**: Long discussions, complex problems



### Implementation Tips



1. **Clear Communication**: Make it obvious when human input is needed

2. **Context Provision**: Show relevant conversation history

3. **Feedback Integration**: Ensure agents properly incorporate human feedback

4. **Timeouts**: In production, add timeouts for approval requests

5. **Audit Trail**: Log all human interventions for compliance



### Production Considerations



For real-world applications:



```csharp

// Async approval with notifications

var reviewId = await _approvalQueue.EnqueueAsync(new ApprovalRequest

{

    Summary = await SummarizeDiscussion(history),

    Priority = Priority.High,

    Timeout = TimeSpan.FromHours(4)

});



// Send notification (email, Slack, Teams)

await _notificationService.NotifyReviewersAsync(reviewId);



// Wait for approval (with timeout and escalation)

var approval = await _approvalQueue.WaitForApprovalAsync(

    reviewId,

    timeout: TimeSpan.FromHours(4)

);

```



🎉 **You now have the tools to add Human-in-the-Loop to any agent orchestration!**